In [1]:
from brainglobe_atlasapi import show_atlases
show_atlases()

╭───────────────────────────────── Brainglobe Atlases ─────────────────────────────────╮
│                                                                           Latest     │
│  Name                              Downloaded  Updated  Local version    version     │
│  allen_mouse_100um                     ✔          ✔          1.2           1.2       │
│  allen_mouse_10um                      ✔          ✔          1.2           1.2       │
│  allen_mouse_25um                      ✔          ✔          1.2           1.2       │
│  allen_mouse_50um                      ✔          ✔          1.2           1.2       │
│  princeton_mouse_20um                  ✔          ✔          1.1           1.1       │
│  whs_sd_rat_39um                       ✔          ✔          1.1           1.1       │
│  example_mouse_100um                                                       1.2       │
│  mpin_zfish_1um                                                            1.0       │
│  allen_human_500um                                                         0.1       │
│  kim_mouse_10um                                                            1.0       │
│  kim_mouse_25um                                                            1.0       │
│  kim_mouse_50um                                                            1.0       │
│  kim_mouse_100um                                                           1.0       │
│  osten_mouse_10um                                                          1.1       │
│  osten_mouse_25um                                                          1.1       │
│  osten_mouse_50um                                                          1.1       │
│  osten_mouse_100um                                                         1.1       │
│  allen_cord_20um                                                           1.0       │
│  azba_zfish_4um                                                            1.1       │
│  perens_lsfm_mouse_20um                                                    1.0       │
│  admba_3d_e11_5_mouse_16um                                                 1.0       │
│  admba_3d_e13_5_mouse_16um                                                 1.0       │
│  admba_3d_e15_5_mouse_16um                                                 1.0       │
│  admba_3d_e18_5_mouse_16um                                                 1.0       │
│  admba_3d_p4_mouse_16.752um                                                1.0       │
│  admba_3d_p14_mouse_16.752um                                               1.0       │
│  admba_3d_p28_mouse_16.752um                                               1.0       │
│  admba_3d_p56_mouse_25um                                                   1.0       │
│  kim_dev_mouse_stp_10um                                                    1.1       │
│  kim_dev_mouse_idisco_10um                                                 1.1       │
│  kim_dev_mouse_mri_a0_10um                                                 1.1       │
│  kim_dev_mouse_mri_adc_10um                                                1.1       │
│  kim_dev_mouse_mri_dwi_10um                                                1.1       │
│  kim_dev_mouse_mri_fa_10um                                                 1.1       │
│  kim_dev_mouse_mri_mtr_10um                                                1.1       │
│  kim_dev_mouse_mri_t2_10um                                                 1.1       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [2]:
from brainatlas2unity.src.brainatlas2unity.conversion import *

## Run pipeline

In [4]:
# RUN
data_path = "./data"

# downloaded = ["allen_mouse_25um","allen_mouse_50um","allen_mouse_100um",
#               "whs_sd_rat_39um",
#               "princeton_mouse_20um"]
# apply_smoothing = [False, False, False,
#                    True,
#                    True]
# fix_normals = [False, False, False,
#                True,
#                True]

downloaded = [
              "whs_sd_rat_39um",
              "princeton_mouse_20um"]
apply_smoothing = [
                   True,
                   True]
fix_normals = [
               True,
               True]
atlases = []

for i, atlas_name in enumerate(downloaded):
    atlas = BrainGlobeAtlas(atlas_name, check_latest=True)
    atlases.append(atlas)

    if not os.path.isdir(os.path.join(data_path,atlas_name)):
        os.mkdir(os.path.join(data_path,atlas_name))
        
    intermediate_meta(atlas, atlas_name, data_path)
    intermediate_structures(atlas, atlas_name, data_path)
    intermediate_ref_image(atlas, atlas_name, data_path)
    intermediate_annot_image(atlas, atlas_name, data_path)
    intermediate_mesh_centers(atlas, atlas_name, data_path)

    # copy files
    copy_mesh_files(atlas, atlas_name, data_path)

    # # mesh handling
    cleanup_folder(atlas_name, data_path)
    # # apply_bg_translation(atlas_name)
    apply_blender_repairs(atlas_name, data_path, normals=fix_normals[i], smoothing=apply_smoothing[i])
    intermediate_mesh_files(atlas, atlas_name, data_path)

KeyboardInterrupt: 

### Replace Waxholm colors

In [4]:
# replace colors in whs_sd_rat_39um with better options

with open('./data/whs_sd_rat_39um/structures.json', 'r') as f: rat_structures = json.load(f)
with open('./data/allen_mouse_25um/structures.json', 'r') as f: mouse_structures = json.load(f)
mouse = {}

for area in mouse_structures:
    mouse[area["acronym"]] = area

rewrite_df = pd.read_csv('waxholm_acronyms_CCF.csv')

def update_json(json_data, acronym, rgb_color):
    for i, area in enumerate(json_data):
        if area["acronym"]==acronym:
            area["rgb_triplet"] = rgb_color
            json_data[i] = area
            return json_data
        
cur_rgb_color = [255, 255, 255]
# go through each area in the dataframe, check if we have a hex code already and apply it, otherwise keep track of the last mouse region and use that color
for i, row in rewrite_df.iterrows():
    ccf_equiv = row["ccf equivalent"]

    if ccf_equiv == "#808080":
        rat_structures = update_json(rat_structures, row["acronym"], [128, 128, 128])
        continue
    if ccf_equiv == "#000000":
        rat_structures = update_json(rat_structures, row["acronym"], [0, 0 , 0])
        continue
    
    if pd.isna(ccf_equiv):
        # if NaN, skip
        rat_structures = update_json(rat_structures, row["acronym"], cur_rgb_color)
    else:    
        # otherwise, save the current value
        cur_rgb_color = mouse[ccf_equiv]['rgb_triplet']
        rat_structures = update_json(rat_structures, row["acronym"], cur_rgb_color)

with open('./data/whs_sd_rat_39um/structures.json', 'w') as f: json.dump(rat_structures, f)


### Waxholm 79
Downsample the waxholm data to 79um instead of 39um

In [9]:
import shutil

source_folder = "./data/whs_sd_rat_39um/"
destination_folder = "./data/whs_sd_rat_78um/"

shutil.copytree(source_folder, destination_folder)

path78 = './data/whs_sd_rat_78um/'

import json
# Load the metadata and change the dimensions
fname = f'{path78}/meta.json'
with open(fname, 'r') as f:
    metadata = json.load(f)
    f.close()

metadata['resolution'] = [78.0, 78.0, 78.0]
metadata['shape'] = [512, 256, 256]

json.dump(metadata, open(fname, 'w'))

# Load the annotation and reference volumes and downsample
import os
import numpy as np

annotation_file = os.path.join(path78,"annotation.bytes")

data_type = np.uintc  # Replace with the actual data type used when saving
shape = (1024, 512, 512)      # Replace with the actual shape of the array

# Load the data
loaded_data = np.fromfile(annotation_file, dtype=data_type)
loaded_data = loaded_data.reshape(shape)  # Reshape the loaded data to its original shape

downsampled_array = loaded_data[::2, ::2, ::2]

downsampled_array.flatten().tofile(annotation_file)


reference_file = os.path.join(path78,"reference.bytes")

data_type = np.float32  # Replace with the actual data type used when saving
shape = (1024, 512, 512)      # Replace with the actual shape of the array

# Load the data
loaded_data = np.fromfile(reference_file, dtype=data_type)
loaded_data = loaded_data.reshape(shape)  # Reshape the loaded data to its original shape

downsampled_array = loaded_data[::2, ::2, ::2]

downsampled_array.flatten().tofile(reference_file)

# Load all the mesh files and decimate them to 10%

import bpy
import os

# Set the path to your folder containing .obj files
folder_path = "C:\proj\VBL\BrainAtlas\Pipelines\data\whs_sd_rat_78um\meshes"

# Set the decimation ratio
decimate_ratio = 0.2

# Get a list of all .obj files in the folder
obj_files = [f for f in os.listdir(folder_path) if f.endswith(".obj")]

# Loop through each .obj file
for obj_file in obj_files:
    # Create a full path to the .obj file
    obj_path = os.path.join(folder_path, obj_file)

    # Load the .obj file into Blender
    bpy.ops.wm.obj_import(filepath=obj_path)

    # Select the imported mesh
    bpy.context.view_layer.objects.active = bpy.context.selected_objects[0]
    bpy.ops.object.mode_set(mode='OBJECT')  # Switch to Object mode
    
    # Add Decimate modifier to the active object
    bpy.ops.object.modifier_add(type='DECIMATE')

    # Set the decimate ratio
    decimate_ratio = 0.5  # Change this value as needed
    bpy.context.object.modifiers["Decimate"].ratio = decimate_ratio

    # Apply the modifier
    bpy.ops.object.modifier_apply(modifier="Decimate")

    # Save the modified mesh
    bpy.ops.wm.obj_export(filepath=obj_path, export_selected_objects =True, export_normals =True, export_materials =False, apply_modifiers = True)

    # Remove the mesh from the scene
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()

print("Decimation complete.")

IndexError: list index out of range